In [1]:
import os
os.chdir("../")

In [2]:
import pandas as pd
import glob
from scipy.stats import ttest_ind

In [3]:
resultDir = 'results'
problem = 'cauctions' # choices=['setcover', 'cauctions', 'facilities', 'indset']
sampling_Strategies = ['uniform5','depthK','depthK2'] # choices: uniform5, depthK, depthK2, depthK3
sampling_seed = 0

In [4]:
sampleTimes_allStrategies = pd.DataFrame()
for sampling_Strategy in sampling_Strategies:
    problem_folders = {
        'setcover': f'setcover/500r_1000c_0.05d({sampling_Strategy})/{sampling_seed}',
        'cauctions': f'cauctions/100_500({sampling_Strategy})/{sampling_seed}',
        'facilities': f'facilities/100_100_5({sampling_Strategy})/{sampling_seed}',
        'indset': f'indset/500_4({sampling_Strategy})/{sampling_seed}',
    }
    problem_folder = problem_folders[problem]
    depthTablePath = f'data/samples/{problem_folder}/depthTable(trainSol).csv'
    depthTable = pd.read_csv(depthTablePath, index_col=0)
    sampleTimes_allStrategies[f'{sampling_Strategy}'] = depthTable['sampleTimes']

FileNotFoundError: [Errno 2] No such file or directory: 'data/samples/setcover/500r_1000c_0.05d(uniform5)/0/depthTable(trainSol).csv'

In [5]:
bin_size = 5
binned = sampleTimes_allStrategies.groupby(sampleTimes_allStrategies.index // bin_size).sum()
binned = binned / binned.sum()
binned['GroupName'] = [f"[{i*bin_size},{(i+1)*bin_size-1}]" for i in binned.index]
binned

TypeError: cannot perform __floordiv__ with this index type: Index

In [6]:
inv_bins = binned.sort_index(ascending=False)
axe = inv_bins.plot.barh()
axe.set_yticklabels(inv_bins['GroupName'])
# axe.savefig(f'depthDist{problem}.pdf')

NameError: name 'binned' is not defined

# 保存单个 depthTable 的 accessTimes

先读取`depthTable(trainSol).csv`, 这个是保存每种sampling strategy采样过程中的访问深度情况和采样情况，我们只需要读取uniform5 的 depthTable 的 accessTime 列就行了


In [10]:
sampleDir = 'data/samples'
problem = 'setcover' # choices=['setcover', 'cauctions', 'facilities', 'indset']
# sampling_Strategy = 'uniform5' # choices: uniform5, depthK, depthK2, depth_adaptive
sampling_seed = 0

depthTablePaths = {
    'setcover': f'setcover/500r_1000c_0.05d(uniform5)/{sampling_seed}',
    'cauctions': f'cauctions/100_500(uniform5)/{sampling_seed}',
    'facilities': f'facilities/100_100_5(uniform5)/{sampling_seed}',
    'indset': f'indset/500_4(uniform5)/{sampling_seed}',
}
depthTablePath = f'{sampleDir}/{depthTablePaths[problem]}/depthTable(trainSol).csv'
depthTable = pd.read_csv(depthTablePath, index_col=0)

In [11]:
bin_size = 5
binned = depthTable['accessTimes'].groupby(depthTable.index // bin_size).sum()
binned /= binned.sum()
binned.index.name = 'depth//5'
binned

depth//5
0    0.086655
1    0.438031
2    0.355058
3    0.110559
4    0.009635
5    0.000061
Name: accessTimes, dtype: float64

In [12]:
binned.to_csv('depthTable_SolSB_binned5.csv')

In [37]:
dff = pd.read_csv('depthTable_SolSB_binned5.csv', index_col='depth//5')
dff

,accessTimes
depth//5,
0,0.126149
1,0.412810
2,0.300835
3,0.105769
4,0.036508
5,0.012833
6,0.004405
7,0.000690
